# Baseball Data Parsing

Splitting stuff by Avery - run this code but don't change it

In [1]:
import pandas as pd
import numpy as np
import math
import unittest

In [2]:
df = pd.read_csv('Stanford_2018_pbp.csv')

In [3]:
# old stuff that might be useful later
# regex_exp = "(?P<batter>.*)" \
#             "(?P<out> fouled out|struck out|grounded out|flied out|popped up" \
#             "|reached on a fielders choice|walked|singled|doubled|tripled|homered)(?:.*)"\
#             "\((?P<count>\d-\d)[ ]?(?P<sequence>\w*)\)(?:.)?" \
#             "(.* (?:advanced to \w+|out at \w+|scored)(?: \w+ to \w+ )?)*"

Parse the player names by splitting on the action verb.

In [4]:
actions = ['fouled', 
           'struck', 
           'grounded', 
           'hit',
           'flied',
           'lined',
           'popped',
           'reached',
           'intentionally',
           'walked',
           'singled',
           'doubled',
           'tripled',
           'homered',
           'advanced',
           'scored',
           'picked',
           'popped',
           'out',
          'failed',
          'stole',
          'pinch']
df_select = df['description'].str.split('|'.join(actions), n=1, expand=True)

#added so regex doesn't confuse . and ,
players = set(i.strip().replace('.', '\.').replace(',', '\,') for i in df_select.dropna()[0] if type(i) == str)
player_name = '|'.join(players)

In [5]:
players

{'ALDERS',
 'ALLEN\\, J\\.',
 'ANDERSON',
 'Acosta\\, M\\.',
 'Aguiar\\, A\\.',
 'Aguiar\\, Alex',
 'Aldrete\\,C',
 'Alvarez\\, A',
 'Amaral',
 'Anderson',
 'Anderson\\, C\\.',
 'Anderson\\, R\\.',
 'Anderson\\, Ryan',
 'Andranovich',
 'Armenta\\, A\\.',
 'Armstrong',
 'Aviles',
 'BARRERA\\, I\\.',
 'BATE\\, B\\.',
 'BECHT',
 'BECHT\\, Joe',
 'BEWLEY\\, B',
 'BOETTO',
 'BOETTO\\, Tony',
 'BRESHEARS\\, C',
 'BRODT',
 'BRODT\\, Jake',
 'BROOKS\\, N\\.',
 'BURDICK',
 'B\\. Borgogno',
 'Baker',
 'Bakst\\,Dan',
 'Bakst\\,Daniel',
 'Barnum\\, B',
 'Beck\\, B\\.',
 'Bellafronto\\, Nick',
 'Berryhill\\, Mitchell',
 'Bertelson',
 'Bishop\\,H',
 'Bissonette',
 'Blas\\, J\\.',
 'Blomgren',
 'Bock',
 'Borgogno\\, Brett',
 'Brady',
 'Branton\\,B',
 'Branton\\,Beau',
 'Briggs\\, C\\.',
 'Briggs\\, Cameron',
 'Brueser\\, N\\.',
 'Brueser\\, Nick',
 'Bullock',
 'Bushor\\, C\\.',
 'CAMPOS\\, M\\.',
 'CLAWSON',
 'COLLARD\\, K\\.',
 'CORDEIRO\\, R\\.',
 'CRESTO',
 'CRESTO\\, John',
 'C\\. Prescott',
 'Ca

Split the descriptions using the player names.

In [6]:
reg_str = '(?P<player1>'+player_name+')'+'(?P<action1>.*?(?='+player_name+'|\.))'\
        +'(?P<player2>'+player_name+')?'+'(?P<action2>.*?(?='+player_name+'|\.))'\
        +'(?P<player3>'+player_name+')?'+'(?P<action3>.*?(?='+player_name+'|\.))'\
        +'(?P<player4>'+player_name+')?'+'(?P<action4>.*?(?='+player_name+'|\.))'\
        +'(?P<player5>'+player_name+')?'+'(?P<action5>.*?(?='+player_name+'|\.))'

df_pinch = df['description']\
                .str.extract('(?P<pinch_player_in>\w+) pinch ran for (?P<pinch_player_out>\w+)', expand=True)

df_splits = pd.concat([df, df['description']\
              .str.extract(reg_str, expand=True), df_pinch], axis=1)\
              .replace(0, np.nan)\
              .replace('', np.nan)
df_splits[['description', \
           'player1', 'action1', \
           'player2', 'action2', \
           'player3', 'action3', \
           'player4', 'action4', \
           'pinch_player_in', 'pinch_player_out']].head(30)


,description,player1,action1,player2,action2,player3,action3,player4,action4,pinch_player_in,pinch_player_out
0,H. LoForte fouled out to lf (3-2 BKBBK).,H. LoForte,fouled out to lf (3-2 BKBBK),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S.Valenzuela struck out swinging (1-2 BKSS).,S.Valenzuela,struck out swinging (1-2 BKSS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R. Cardenas fouled out to c (1-1 KB).,R. Cardenas,fouled out to c (1-1 KB),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Hoerner,N grounded out to ss (3-1 KBBB).","Hoerner,N",grounded out to ss (3-1 KBBB),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Handley, M. popped up to 2b (2-2 BKBS).","Handley, M.",popped up to 2b (2-2 BKBS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Bakst,Dan flied out to cf (2-1 KBB).","Bakst,Dan",flied out to cf (2-1 KBB),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C. Prescott grounded out to 3b (1-2 BSK).,C. Prescott,grounded out to 3b (1-2 BSK),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,D. Cope struck out swinging (2-2 BFBKFS).,D. Cope,struck out swinging (2-2 BFBKFS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,T. Lasch popped up to ss (1-0 B).,T. Lasch,popped up to ss (1-0 B),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Wulff, B. walked (3-0 BBBB).","Wulff, B.",walked (3-0 BBBB),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Parse each action and assign each player to a base.

In [7]:
def label_bases_all(df_splits):
    def label_end_base(row, player_id):
        if type(row[f'player{player_id}']) == float and math.isnan(row[f'player{player_id}']):
            return np.nan
        if type(row[f'action{player_id}']) == float and math.isnan(row[f'action{player_id}']):
            return "unknown"
        if row[f'player{player_id}_out']:
            return "out"
        elif row[f'player{player_id}_home']:
            return "score"
        elif row[f'player{player_id}_third']:
            return "third"
        elif row[f'player{player_id}_second']:
            return "second"
        elif row[f'player{player_id}_first']:
            return "first"
        else:
            return "unknown"
    
    for i in [1, 2, 3, 4]:
        df_splits[f'player{i}_out'] = df_splits[f'action{i}'].str.contains('out|popped|double play|infield')
        df_splits[f'player{i}_first'] = df_splits[f'action{i}'].str.contains('singled|walked|reached|hit by pitch')
        df_splits[f'player{i}_second'] = df_splits[f'action{i}'].str.contains('doubled|second')
        df_splits[f'player{i}_third'] = df_splits[f'action{i}'].str.contains('tripled|third')
        df_splits[f'player{i}_home'] = df_splits[f'action{i}'].str.contains('scored|homered')
        df_splits[f'player{i}_end'] = df_splits.apply (lambda row: label_end_base(row, i), axis=1)
        

In [8]:
def add_ids(df_splits):
    current_game_id, current_event_id = 0, 0
    game_ids, event_ids = [], []

    for i, row in df_splits.iterrows():
        if current_event_id > 30 and row['inning'] == 1:
            current_game_id += 1
            current_event_id = 0

        game_ids.append(current_game_id)
        event_ids.append(current_event_id)
        current_event_id += 1

    df_splits['game_id'] = game_ids
    df_splits['event_id'] = event_ids


In [20]:
label_bases_all(df_splits)
add_ids(df_splits)

df_splits[['game_id', 'event_id', 'inning', 'inning_top_bot', 'score', 'batting', 'fielding', 'description',\
            'player1', 'player1_end', \
           'player2', 'player2_end', \
           'player3', 'player3_end', \
           'player4', 'player4_end',\
           'pinch_player_in', 'pinch_player_out']].loc[861:867]

,game_id,event_id,inning,inning_top_bot,score,batting,fielding,description,player1,player1_end,player2,player2_end,player3,player3_end,player4,player4_end,pinch_player_in,pinch_player_out
861,9,76,8,top,4-0,Michigan,Stanford,"Brueser, N. to 1b for Daschbach.","Brueser, N.",unknown,Daschbach,unknown,NaN,NaN,NaN,NaN,NaN,NaN
862,9,77,8,top,4-0,Michigan,Stanford,Lewis reached on a throwing error by ss (1-1 BK).,Lewis,first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863,9,78,8,top,4-0,Michigan,Stanford,Chavez singled to right field (0-1 K); Lewis a...,Chavez,first,Lewis,third,NaN,NaN,NaN,NaN,NaN,NaN
864,9,79,8,top,4-0,Michigan,Stanford,Nwogu pinch ran for Chavez.,Nwogu,unknown,Chavez,unknown,NaN,NaN,NaN,NaN,Nwogu,Chavez
865,9,80,8,top,5-0,Michigan,Stanford,Keener grounded into double play 2b to ss to 1...,Keener,out,Nwogu,out,Lewis,score,NaN,NaN,NaN,NaN
866,9,81,8,top,5-0,Michigan,Stanford,Donovan popped up to ss (1-2 FBK).,Donovan,out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
867,9,82,8,bot,5-0,Stanford,Michigan,"Carter, B. pinch hit for Handley, M..","Carter, B.",unknown,"Handley, M.",unknown,NaN,NaN,NaN,NaN,NaN,NaN


A class to package all the data of a row, so Donnie doesn't have to deal with the pandas mess above. 

In [10]:
class BaseballPlay:      
    def __init__(self, row):
        self.event_id = row['event_id']
        self.game_id = row['game_id']
        self.date = row['date']
        self.location = row['location']
        self.inning = row['inning']
        self.inning_top_bot = row['inning_top_bot']
        self.score = row['score']
        self.batting = row['batting']
        self.fielding = row['fielding']
        self.description = row['description']
        
        if type(row['pinch_player_in']) == str:
            self.pinch_runner = [row['pinch_player_out'], row['pinch_player_in']]
        else:
            self.pinch_runner = None
        
        self.updates = []
        for i in [1, 2, 3, 4]:
            if type(row[f'player{i}']) == str:
                self.updates.append((row[f'player{i}'], row[f'player{i}_end']))
                
        self.first_base_before = None 
        self.second_base_before = None
        self.third_base_before = None
        self.outs_before = None
        
        self.first_base_after = None 
        self.second_base_after = None
        self.third_base_after = None
        self.outs_after = None
        
        self.runs_in_inning = None
        self.runs_in_inning_so_far_before = None
        self.runs_in_inning_so_far_after = None
        
    def get_id(self):
        return (self.game_id, self.event_id)

    def get_inning(self):
        return (self.inning, self.inning_top_bot)
    
    def set_bases_before(self, first=None, second=None, third=None):
        self.first_base_before = first
        self.second_base_before = second
        self.third_base_before = third
        
    def set_bases_after(self, first=None, second=None, third=None):
        self.first_base_after = first
        self.second_base_after = second
        self.third_base_after = third
        
    def set_outs(self, before, after):
        self.outs_before = before
        self.outs_after = after
        
    def set_runs(self, total, before, after):
        self.runs_in_inning = total
        self.runs_in_inning_so_far_before = before
        self.runs_in_inning_so_far_after = after
    
    # print functions so you can print a BaseballPlay
    def __str__(self):
        return f"({self.description}) \n       G{self.game_id}E{self.event_id} {self.inning_top_bot}{self.inning} {self.outs_after} outs "\
        f"[1B:{self.first_base_after} 2B:{self.second_base_after} 3B:{self.third_base_after}]" \
        f" {self.runs_in_inning_so_far_after} runs"
        
        

In [11]:
all_plays = [BaseballPlay(row) for i, row in df_splits.iterrows()]
print(len(all_plays))

5446


# Game Reconstruction
The 'all_plays' list contains a bunch of BaseballPlays, and each of these has the information about one play (one row in that table above). Use the information in the updates to reconstruct the game and record who is at each base before and after the play, and how many outs and runs there are.

In [12]:
print("Before with no hard-code")
print(all_plays[9])

# This is a hard coded way, but you'll want to create something more generic.
all_plays[9].set_bases_after(first='Wulff, B.')
all_plays[9].set_outs(before=0, after=0)
all_plays[9].set_runs(total=1, before=0, after=0)

print("After with hard-code")
print(all_plays[9])

Before with no hard-code
(Wulff, B. walked (3-0 BBBB).) 
       G0E9 bot2 None outs [1B:None 2B:None 3B:None] None runs
After with hard-code
(Wulff, B. walked (3-0 BBBB).) 
       G0E9 bot2 0 outs [1B:Wulff, B. 2B:None 3B:None] 0 runs


In [13]:
# Be aware of pinch runners! You can check if a play was actually just a pinch runner being subbed in.
print(all_plays[864])
print(all_plays[864].pinch_runner)

# Be aware of pinch runners! You can check if a play was actually just a pinch runner being subbed in.
print(all_plays[865])
print(all_plays[865].pinch_runner)

(Nwogu pinch ran for Chavez.) 
       G9E79 top8 None outs [1B:None 2B:None 3B:None] None runs
['Chavez', 'Nwogu']
(Keener grounded into double play 2b to ss to 1b (1-1 BK); Nwogu out on the play; Lewis scored, unearned.) 
       G9E80 top8 None outs [1B:None 2B:None 3B:None] None runs
None


In [14]:
# TODO: write code that goes through all updates one at a time
# and records who is at each base before and after this play
# also update the number of outs and runs before and after the play
# remember you can get the inning/top-bot in the fields inside all_plays[i]

# Unit Tests
I wrote a couple of these unit tests so you can quickly confirm the behavior.
They're not exhaustive and I wrote these really quickly so no guarantees they're perfect.
Double check them yourself.

In [15]:
# This ones a pretty standard use case. A couple walks, singles, runs, and outs.

class TestG0E9Bot2(unittest.TestCase):

    def test_event_9(self):
        # 9 [('Wulff, B.', 'first')]
        print(all_plays[9])
        self.assertTrue(all_plays[9].first_base_before == None)
        self.assertTrue(all_plays[9].second_base_before == None)
        self.assertTrue(all_plays[9].third_base_before == None)
        self.assertTrue(all_plays[9].outs_before == 0)

        self.assertTrue(all_plays[9].first_base_after == 'Wulff, B.')
        self.assertTrue(all_plays[9].second_base_after == None)
        self.assertTrue(all_plays[9].third_base_after == None)
        self.assertTrue(all_plays[9].outs_after == 0)

        self.assertTrue(all_plays[9].runs_in_inning == 1)
        self.assertTrue(all_plays[9].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[9].runs_in_inning_so_far_after == 0)
        
    def test_event_10(self):
        # 10 [('Daschbach', 'first'), ('Wulff, B.', 'second')]
        print(all_plays[10])
        self.assertTrue(all_plays[10].first_base_before == 'Wulff, B.')
        self.assertTrue(all_plays[10].second_base_before == None)
        self.assertTrue(all_plays[10].third_base_before == None)
        self.assertTrue(all_plays[10].outs_before == 0)

        self.assertTrue(all_plays[10].first_base_after == 'Daschbach')
        self.assertTrue(all_plays[10].second_base_after == 'Wulff, B.')
        self.assertTrue(all_plays[10].third_base_after == None)
        self.assertTrue(all_plays[10].outs_after == 0)

        self.assertTrue(all_plays[10].runs_in_inning == 1)
        self.assertTrue(all_plays[10].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[10].runs_in_inning_so_far_after == 0)
        
    def test_event_11(self):
        # 11 [('Stowers,K', 'first'), ('Daschbach', 'out'), ('Wulff, B.', 'third')]
        print(all_plays[11])
        self.assertTrue(all_plays[11].first_base_before == 'Daschbach')
        self.assertTrue(all_plays[11].second_base_before == 'Wulff, B.')
        self.assertTrue(all_plays[11].third_base_before == None)
        self.assertTrue(all_plays[11].outs_before == 0)

        self.assertTrue(all_plays[11].first_base_after == 'Stowers,K')
        self.assertTrue(all_plays[11].second_base_after == None)
        self.assertTrue(all_plays[11].third_base_after == 'Wulff, B.')
        self.assertTrue(all_plays[11].outs_after == 1)

        self.assertTrue(all_plays[11].runs_in_inning == 1)
        self.assertTrue(all_plays[11].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[11].runs_in_inning_so_far_after == 0)
        
    def test_event_12(self):
        # 12 [('Wilson, A.', 'out')]
        print(all_plays[12])
        self.assertTrue(all_plays[12].first_base_before == 'Stowers,K')
        self.assertTrue(all_plays[12].second_base_before == None)
        self.assertTrue(all_plays[12].third_base_before == 'Wulff, B.')
        self.assertTrue(all_plays[12].outs_before == 1)

        self.assertTrue(all_plays[12].first_base_after == 'Stowers,K')
        self.assertTrue(all_plays[12].second_base_after == None)
        self.assertTrue(all_plays[12].third_base_after == 'Wulff, B.')
        self.assertTrue(all_plays[12].outs_after == 2)

        self.assertTrue(all_plays[12].runs_in_inning == 1)
        self.assertTrue(all_plays[12].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[12].runs_in_inning_so_far_after == 0)
        
    def test_event_13(self):
        # 13 [('Tawa, T.', 'first'), ('Stowers,K', 'second'), ('Wulff, B.', 'score')]
        print(all_plays[13])
        self.assertTrue(all_plays[13].first_base_before == 'Stowers,K')
        self.assertTrue(all_plays[13].second_base_before == None)
        self.assertTrue(all_plays[13].third_base_before == 'Wulff, B.')
        self.assertTrue(all_plays[13].outs_before == 0)

        self.assertTrue(all_plays[13].first_base_after == 'Tawa, T.')
        self.assertTrue(all_plays[13].second_base_after == 'Stowers,K')
        self.assertTrue(all_plays[13].third_base_after == None)
        self.assertTrue(all_plays[13].outs_after == 2)

        self.assertTrue(all_plays[13].runs_in_inning == 1)
        self.assertTrue(all_plays[13].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[13].runs_in_inning_so_far_after == 1)

    def test_event_14(self):
        # 14 [('Kuet,J', 'out')]
        print(all_plays[14])
        self.assertTrue(all_plays[14].first_base_before == 'Tawa, T.')
        self.assertTrue(all_plays[14].second_base_before == 'Stowers,K')
        self.assertTrue(all_plays[14].third_base_before == None)
        self.assertTrue(all_plays[14].outs_before == 2)

        self.assertTrue(all_plays[14].first_base_after == None)
        self.assertTrue(all_plays[14].second_base_after == None)
        self.assertTrue(all_plays[14].third_base_after == None)
        self.assertTrue(all_plays[14].outs_after == 3)

        self.assertTrue(all_plays[14].runs_in_inning == 1)
        self.assertTrue(all_plays[14].runs_in_inning_so_far_before == 1)
        self.assertTrue(all_plays[14].runs_in_inning_so_far_after == 1)

In [16]:
# This ones has a few pinch runners and a double play.

class TestG9E76Top8(unittest.TestCase):

    def test_event_76(self):
        # 861, pinch runner, nothing happens
        print(all_plays[861])
        self.assertTrue(all_plays[861].first_base_before == None)
        self.assertTrue(all_plays[861].second_base_before == None)
        self.assertTrue(all_plays[861].third_base_before == None)
        self.assertTrue(all_plays[861].outs_before == 0)

        self.assertTrue(all_plays[861].first_base_after == None)
        self.assertTrue(all_plays[861].second_base_after == None)
        self.assertTrue(all_plays[861].third_base_after == None)
        self.assertTrue(all_plays[861].outs_after == 0)

        self.assertTrue(all_plays[861].runs_in_inning == 1)
        self.assertTrue(all_plays[861].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[861].runs_in_inning_so_far_after == 0)
        
    def test_event_77(self):
        # 862, Lewis to first
        print(all_plays[862])
        self.assertTrue(all_plays[862].first_base_before == None)
        self.assertTrue(all_plays[862].second_base_before == None)
        self.assertTrue(all_plays[862].third_base_before == None)
        self.assertTrue(all_plays[862].outs_before == 0)

        self.assertTrue(all_plays[862].first_base_after == 'Lewis')
        self.assertTrue(all_plays[862].second_base_after == None)
        self.assertTrue(all_plays[862].third_base_after == None)
        self.assertTrue(all_plays[862].outs_after == 0)

        self.assertTrue(all_plays[862].runs_in_inning == 1)
        self.assertTrue(all_plays[862].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[862].runs_in_inning_so_far_after == 0)
        
    def test_event_78(self):
        # 863, Lewis to third, Chavez to first
        print(all_plays[863])
        self.assertTrue(all_plays[863].first_base_before == 'Lewis')
        self.assertTrue(all_plays[863].second_base_before == None)
        self.assertTrue(all_plays[863].third_base_before == None)
        self.assertTrue(all_plays[863].outs_before == 0)

        self.assertTrue(all_plays[863].first_base_after == 'Chavez')
        self.assertTrue(all_plays[863].second_base_after == None)
        self.assertTrue(all_plays[863].third_base_after == 'Lewis')
        self.assertTrue(all_plays[863].outs_after == 0)

        self.assertTrue(all_plays[863].runs_in_inning == 1)
        self.assertTrue(all_plays[863].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[863].runs_in_inning_so_far_after == 0)
        
    def test_event_79(self):
        # 864, pinch runner
        print(all_plays[864])
        self.assertTrue(all_plays[864].first_base_before == 'Chavez')
        self.assertTrue(all_plays[864].second_base_before == None)
        self.assertTrue(all_plays[864].third_base_before == 'Lewis')
        self.assertTrue(all_plays[864].outs_before == 0)

        self.assertTrue(all_plays[864].first_base_after == 'Nwogu')
        self.assertTrue(all_plays[864].second_base_after == None)
        self.assertTrue(all_plays[864].third_base_after == 'Lewis')
        self.assertTrue(all_plays[864].outs_after == 0)

        self.assertTrue(all_plays[864].runs_in_inning == 1)
        self.assertTrue(all_plays[864].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[864].runs_in_inning_so_far_after == 0)
        
    def test_event_80(self):
        # 865, Keener hits into double play, Nwogu out, Lewis scores
        print(all_plays[865])
        self.assertTrue(all_plays[865].first_base_before == 'Nwogu')
        self.assertTrue(all_plays[865].second_base_before == None)
        self.assertTrue(all_plays[865].third_base_before == 'Lewis')
        self.assertTrue(all_plays[865].outs_before == 0)

        self.assertTrue(all_plays[865].first_base_after == None)
        self.assertTrue(all_plays[865].second_base_after == None)
        self.assertTrue(all_plays[865].third_base_after == None)
        self.assertTrue(all_plays[865].outs_after == 2)

        self.assertTrue(all_plays[865].runs_in_inning == 1)
        self.assertTrue(all_plays[865].runs_in_inning_so_far_before == 0)
        self.assertTrue(all_plays[865].runs_in_inning_so_far_after == 1)
        
    def test_event_81(self):
        # 866, Donovan out
        print(all_plays[866])
        self.assertTrue(all_plays[866].first_base_before == None)
        self.assertTrue(all_plays[866].second_base_before == None)
        self.assertTrue(all_plays[866].third_base_before == None)
        self.assertTrue(all_plays[866].outs_before == 2)

        self.assertTrue(all_plays[866].first_base_after == None)
        self.assertTrue(all_plays[866].second_base_after == None)
        self.assertTrue(all_plays[866].third_base_after == None)
        self.assertTrue(all_plays[866].outs_after == 3)

        self.assertTrue(all_plays[866].runs_in_inning == 1)
        self.assertTrue(all_plays[866].runs_in_inning_so_far_before == 1)
        self.assertTrue(all_plays[866].runs_in_inning_so_far_after == 1)

In [17]:
# Run this cell to run the unit tests. The first test passes only because we hard-coded it above.
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

FFFFF.FFFFFF

(Daschbach singled through the left side (1-1 FB) Wulff, B. advanced to second.) 
       G0E10 bot2 None outs [1B:None 2B:None 3B:None] None runs
(Stowers,K reached on a fielders choice (0-0) Daschbach out at second ss to 2b Wulff, B. advanced to third.) 
       G0E11 bot2 None outs [1B:None 2B:None 3B:None] None runs
(Wilson, A. popped up to ss (2-1 BBF).) 
       G0E12 bot2 None outs [1B:None 2B:None 3B:None] None runs
(Tawa, T. singled through the left side, RBI (0-2 KS) Stowers,K advanced to second Wulff, B. scored.) 
       G0E13 bot2 None outs [1B:None 2B:None 3B:None] None runs
(Kuet,J struck out swinging (2-2 BFSBFFFS).) 
       G0E14 bot2 None outs [1B:None 2B:None 3B:None] None runs
(Wulff, B. walked (3-0 BBBB).) 
       G0E9 bot2 0 outs [1B:Wulff, B. 2B:None 3B:None] 0 runs
(Brueser, N. to 1b for Daschbach.) 
       G9E76 top8 None outs [1B:None 2B:None 3B:None] None runs
(Lewis reached on a throwing error by ss (1-1 BK).) 
       G9E77 top8 None outs [1B:None 2B:None 3B:Non


FAIL: test_event_10 (__main__.TestG0E9Bot2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-15-dff939935f71>", line 25, in test_event_10
    self.assertTrue(all_plays[10].first_base_before == 'Wulff, B.')
AssertionError: False is not true

FAIL: test_event_11 (__main__.TestG0E9Bot2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-15-dff939935f71>", line 42, in test_event_11
    self.assertTrue(all_plays[11].first_base_before == 'Daschbach')
AssertionError: False is not true

FAIL: test_event_12 (__main__.TestG0E9Bot2)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-15-dff939935f71>", line 59, in test_event_12
    self.assertTrue(all_plays[12].first_base_before == 'Stowers,K')
AssertionError: False is not true

FAIL: test_event_13 (__main__.TestG0